In [14]:
import json
import time
import azureml
import logging
from azureml.core.model import Model
from azureml.core import Workspace, Run, Datastore, Experiment
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.train.hyperdrive import *
from azureml.train.estimator import Estimator
from azureml.widgets import RunDetails

print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.76


In [2]:
# use this code to set up config file
subscription_id =''
resource_group =''
workspace_name = ''

try:
   ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
   ws.write_config()
   print('Workspace configuration succeeded. You are all set!')
except:
   print('Workspace not found. TOO MANY ISSUES!!!')
ws = Workspace.from_config()

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


Workspace configuration succeeded. You are all set!


# Compute Environment
Update the cluster to the existing compute name or if there is not an existing compute it will create one for you with the provisioning_configuration specificed.

In [9]:
cluster = 'trainingCompute'
try:
    compute = ComputeTarget(workspace=ws, name=cluster)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC12', min_nodes=1, max_nodes=5)
    compute = ComputeTarget.create(ws, cluster, compute_config)
    compute.wait_for_completion(show_output=True)

Found existing compute target


## Manage data 

In [3]:
from six.moves.urllib.request import urlretrieve
from pathlib import Path

data_path = 'data'
mnist_file = 'https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz'
target = Path(data_path).joinpath('mnist.npz')
if not os.path.exists(target):
    os.mkdir(data_path)
    print('downloading {} ...'.format(mnist_file))
    file, output = urlretrieve(mnist_file, target)

downloading https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz ...


In [5]:
ds = ws.get_default_datastore()
mnist_data = ds.upload(src_dir = 'data', target_path = 'mnist', show_progress = True)

Uploading an estimated of 1 files
Target already exists. Skipping upload for mnist/mnist.npz
Uploaded 0 files


# Optimize hyperparameters 

In [11]:
# same as above but increase the max_steps and remove the parameters
script_params={
    '--data': mnist_data,
    '--epochs': 100
}

mnist = Experiment(ws, 'mnist_lab')
estimator = Estimator(source_directory='.',
                      compute_target=compute,
                      entry_script='train.py',
                      use_gpu=True,
                      pip_packages=['matplotlib', 'pathlib', 'tensorflow-gpu'])

'gpu_support' is no longer necessary; AzureML now automatically detects and uses nvidia docker extension when it is available. It will be removed in a future release.
'gpu_support' is no longer necessary; AzureML now automatically detects and uses nvidia docker extension when it is available. It will be removed in a future release.


## Hyper parameter search

In [16]:
ps = RandomParameterSampling(
    {
        '--layer': choice(128, 512, 1024),
        '--lr': loguniform(-15, -3),
        '--batch': choice(16, 32, 64, 128)
    })

early_termination_policy = BanditPolicy(slack_factor = 0.15, evaluation_interval=2)

hyperdrive_run_config = HyperDriveConfig(estimator = estimator, 
                                            hyperparameter_sampling = ps, 
                                            policy = early_termination_policy,
                                            primary_metric_name = "accuracy",
                                            primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                            max_total_runs = 10,
                                            max_concurrent_runs = 5)

hd_run = mnist.submit(hyperdrive_run_config)

RunDetails(hd_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'NOTSE…